In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam







In [2]:
# 하이퍼파라미터 정의
batch_size = 50
num_classes = 62
epochs = 10
learning_rate = 0.0001
input_shape = (28, 28, 1)

In [3]:
# CSV 파일 로드
train_df = pd.read_csv("emnist-balanced-train.csv", header=None)
test_df = pd.read_csv("emnist-balanced-test.csv", header=None)


FileNotFoundError: [Errno 2] No such file or directory: 'emnist-balanced-train.csv'

In [60]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18795,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18796,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18797,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18798,26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


from matplotlib import pyplot as plt
test_df[0].plot(kind='hist', bins=20, title=0)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
test_df[5].plot(kind='hist', bins=20, title=5)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
test_df[6].plot(kind='hist', bins=20, title=6)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
test_df[7].plot(kind='hist', bins=20, title=7)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series[1]
  ys = series[7]
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = test_df.sort_values(1, ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel(1)
_ = plt.ylabel(7)

from matplotlib import pyplot as plt
test_df[0].plot(kind='line', figsize=(8, 4), title=0)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
test_df[5].plot(kind='line', figsize=(8, 4), title=5)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
test_df[6].plot(kind='line', figsize=(8, 4), title=6)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
test_df[7].plot(kind='line', figsize=(8, 4), title=7)
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# 첫 번째 열은 라벨, 나머지 열은 이미지 픽셀 데이터
X_train = train_df.iloc[:, 1:].values
y_train = train_df.iloc[:, 0].values
X_test = test_df.iloc[:, 1:].values
y_test = test_df.iloc[:, 0].values

In [4]:
y_test

NameError: name 'y_test' is not defined

In [73]:


# X_train과 X_test를 28x28x1 형태로 변환 (784개의 픽셀을 28x28 이미지로 재구성)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [63]:

# X_train과 X_test를 0~1 사이 값으로 정규화
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [74]:
# y_train과 y_test를 one-hot 인코딩 (정수형 클래스 라벨을 원-핫 벡터로 변환)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [75]:
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (31914, 62)
y_test shape: (18800, 62)


In [79]:
# X_train과 X_test를 2D로 변환하여 스케일링 적용
num_samples_train = X_train.shape[0]
num_samples_test = X_test.shape[0]

# 28x28x1 이미지를 784개의 벡터로 변환
X_train_reshaped = X_train.reshape(num_samples_train, -1)
X_test_reshaped = X_test.reshape(num_samples_test, -1)

# StandardScaler 적용
sc = StandardScaler()
X_train_reshaped = sc.fit_transform(X_train_reshaped)
X_test_reshaped = sc.transform(X_test_reshaped)

# 다시 원래 차원으로 변환
X_train = X_train_reshaped.reshape(num_samples_train, 28, 28, 1)
X_test = X_test_reshaped.reshape(num_samples_test, 28, 28, 1)

print(X_train.shape)  # (samples, 28, 28, 1)
print(X_test.shape)   # (samples, 28, 28, 1)

(31914, 28, 28, 1)
(18800, 28, 28, 1)


In [80]:

# CNN 모델 정의
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(120, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(360, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [81]:

# 모델 컴파일
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping 설정 (과적합 방지)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 저장 체크포인트 설정 (가장 좋은 성능의 모델을 저장)
checkpoint = ModelCheckpoint('best_emnist_model.keras', save_best_only=True, monitor='val_loss')

# 모델 훈련
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    validation_data=(X_test, y_test),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=epochs,
                    callbacks=[checkpoint, early_stopping])

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


638/638 ━━━━━━━━━━━━━━━━━━━━ 30s 35ms/step - accuracy: 0.0277 - loss: nan - val_accuracy: 0.0213 - val_loss: nan
Epoch 2/10
  1/638 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0000e+00 - loss: nan

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


638/638 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0213 - val_loss: nan
Epoch 3/10
638/638 ━━━━━━━━━━━━━━━━━━━━ 29s 27ms/step - accuracy: 0.0235 - loss: nan - val_accuracy: 0.0213 - val_loss: nan
Epoch 4/10
638/638 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0213 - val_loss: nan
Epoch 5/10
638/638 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - accuracy: 0.0218 - loss: nan - val_accuracy: 0.0213 - val_loss: nan


In [54]:
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import base64
from io import BytesIO

# EMNIST byclass 데이터셋은 62개의 클래스(숫자 0-9, 대문자 A-Z, 소문자 a-z)를 포함함
emnist_labels = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',  # 숫자 0-9
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',  # 대문자 A-Z
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'   # 소문자 a-z
]

# 저장된 CNN 모델 불러오기
model = load_model('best_emnist_model.keras')
print("모델이 성공적으로 로드되었습니다.")

# 이미지 전처리 함수
def preprocess_image(image_path):
    # 이미지를 불러와서 흑백으로 변환
    image = Image.open(image_path).convert('L')

    # 이미지를 28x28 픽셀로 리사이즈
    image = image.resize((28, 28))

    # numpy 배열로 변환하고 모델 입력 형태로 재구성
    image_array = np.array(image).reshape(1, 28, 28, 1).astype('float32') / 255.0

    print(f"이미지 배열 크기: {image_array.shape}")
    return image_array

# 예측 함수
def predict_image(image_path):
    # 이미지 전처리
    processed_image = preprocess_image(image_path)

    # 예측 실행
    prediction = model.predict(processed_image)

    # 가장 높은 확률의 클래스 인덱스 찾기
    predicted_class_index = np.argmax(prediction)

    # 해당 인덱스를 EMNIST 라벨로 변환
    predicted_character = emnist_labels[predicted_class_index]

    print(f"예측된 문자: {predicted_character}")
    return predicted_character

# 예시 이미지 경로
image_path = 'testfive.PNG'  # 여기에 예측할 이미지 경로를 입력하세요
predict_image(image_path)

ValueError: File not found: filepath=best_emnist_model.keras. Please ensure the file is an accessible `.keras` zip file.